# A/B Testing

In [ ]:
import numpy as np
import pandas as pd
from scipy.special import comb
from scipy import stats
import seaborn as sns

## Outcomes

- Describe what an A/B Test is:
    - When it is used
    - What forms it may take
- Conduct an A/B Test in Python using `scipy`


A/B Testing is really just a form of hypothesis testing applied to a business problem. And so it can take [many forms](https://en.wikipedia.org/wiki/A/B_testing).

The classic form of A/B Testing is exposing customers to two different versions of a website (the A and B versions) and then conducting a hypothesis test to see if their behavior is significantly different between the two versions.

Even limited to this context, there are [many](https://neilpatel.com/blog/19-obvious-ab-tests/) tests one might run!

We'll try a couple examples of A/B Testing here.

To start we'll try two examples with fake data:

## Example 1: Canadian Winter Temperatures

In [ ]:
a1 = pd.read_csv('fake_data/a1.csv', usecols=['target'])
b1 = pd.read_csv('fake_data/b1.csv', usecols=['target'])

In [ ]:
a1.head()

In [ ]:
b1.head()

### Kernel Density Plots

In [ ]:
sns.kdeplot(a1['target']);

In [ ]:
sns.kdeplot(b1['target']);

### Hypothesis Test

The data here represent average winter temperatures at various locations in Canada, and the question is whether the temperatures in Group A are lower than temperatures in Group B.

**Null hypothesis**: The temperatures in Group A are not lower than the temperatures in Group B.

**Alternative hypothesis**: The temperatures in Group A are lower than the temperatures in Group B.

### By Hand
First let's try this by hand, assuming a $t$-distribution, and setting an $\alpha$ threshold of 0.02:

We'll start by calculating the pooled variance:
- $\Large s^2_P=\frac{(n_1-1)s^2_1 - (n_2-1)s^2_2}{n_1+n_2-2}$.

In [ ]:
var_pooled = ((len(a1)-1) * a1.var() - (len(b1)-1) * b1.var()) / (len(a1) + len(b1) - 2)
var_pooled

Now we calculate the $t$-statistic:
- $\Large t=\frac{\bar{x_1} - \bar{x_2}}{s^2_P\sqrt{\frac{1}{n_1} + \frac{1}{n_2}}}$

In [ ]:
t = (a1.mean() - b1.mean()) / (var_pooled * np.sqrt(1/len(a1) + 1/len(b1)))
t

And the critical $t$-stat is:

In [ ]:
stats.t.ppf(0.99, df=len(a1)+len(b1)-2)

### In One Line!

In [ ]:
stats.ttest_ind(a1, b1, equal_var=False).pvalue

## Example 2: Online Sales

Now let's try a binomial A/B Test (where the variable of interest is binary). We can use [Fisher's exact test](https://en.wikipedia.org/wiki/Fisher%27s_exact_test).

### Question

We have data about whether customers completed sales transactions, segregated by the type of ad banners to which the customers were exposed.

The question we want to answer is whether there was any difference in sales "conversions" between desktop customers who saw the sneakers banner and desktop customers who saw the accessories banner in the month of May 2019.

### Getting the Data

First let's download the data from [kaggle](https://www.kaggle.com/podsyp/how-to-do-product-analytics).

In [ ]:
!pwd

In [ ]:
!unzip /Users/dbraslow/Downloads/archive.zip

In [ ]:
!mkdir data

In [ ]:
!ls

In [ ]:
!mv /Users/dbraslow/Downloads/product.csv data

Let's go ahead and amend the `.gitignore` file now so that we don't accidentally add the data to our next commit.

In [ ]:
!(echo ""; echo "# data"; echo "product.csv") >> .gitignore

In [ ]:
df = pd.read_csv('data/product.csv')

In [ ]:
df.head()

In [ ]:
df.info()

### EDA

Lets's look at the different banner types:

In [ ]:
df['product'].value_counts()

In [ ]:
df.groupby('product')['target'].value_counts()

Let's look at the range of time-stamps on these data:

In [ ]:
df['time'].min()

In [ ]:
df['time'].max()

Let's check the counts of the different site_version values:

In [ ]:
df['site_version'].value_counts()

In [ ]:
df['title'].value_counts()

In [ ]:
df.groupby('title').agg({'target': 'mean'})

### Experimental Setup

We need to filter by site_version, time, and product:

In [ ]:
df_AB = df[(df['site_version'] == 'desktop') &
           (df['time'] >= '2019-05-01') &
           ((df['product'] == 'accessories') | (df['product'] == 'sneakers'))].reset_index(drop = True)

In [ ]:
df_AB.tail()

### The Hypotheses

NULL: Customers who saw the sneakers banner were no more or less likely to buy than customers who saw the accessories banner.

ALTERNATIVE: Customers who saw the sneakers banner were more or less likely to buy than customers who saw the accessories banner.

### Setting a Threshold

We'll set a false-positive rate of $\alpha = 0.05$.

### Preparing Fisher's Test

Fisher's Test is an exact calculation of a $p$-value that requires four quantities: the respective numbers of 1's and 0's for each class.

In [ ]:
df_A = df_AB[df_AB['product'] == 'accessories']
df_B = df_AB[df_AB['product'] == 'sneakers']

We calculate values in a 2x2 table: the numbers of people who did or did not submit orders, both for the accessories banner and thesneakers banner. 

In [ ]:
accessories_orders = sum(df_A['target'])
sneakers_orders = sum(df_B['target'])

accessories_orders, sneakers_orders

To get the numbers of people who didn't submit orders, we get the total number of people who were shown banners and then subtract the numbers of people who did make orders.

In [ ]:
accessories_total = sum(df_A['title'] == 'banner_show')
sneakers_total = sum(df_B['title'] == 'banner_show')

accessories_no_orders = accessories_total - accessories_orders
sneakers_no_orders = sneakers_total - sneakers_orders

accessories_no_orders, sneakers_no_orders

### Calculation

Fisher's Test tells us that the $p$-value corresponding to our distribution is given by:

$\Large p = \frac{(a+b)!(c+d)!(a+c)!(b+d)!}{a!b!c!d!n!}$

In [ ]:
a = accessories_orders
b = sneakers_orders
c = accessories_no_orders
d = sneakers_no_orders

In [ ]:
ab_choose_a = comb(a+b, a, exact=True)

In [ ]:
cd_choose_c = comb(c+d, c, exact=True)

In [ ]:
n_choose_ac = comb(a+b+c+d, a+c, exact=True)

In [ ]:
p = ab_choose_a * cd_choose_c / n_choose_ac
p

Comparing with `stats.fisher_exact()`:

In [ ]:
stats.fisher_exact(np.array([[a, b], [c, d]]))

This extremely low $p$-value suggests that these two groups are genuinely performing differently. In particular, the desktop customers who saw the sneakers banner in May 2019 bought at a higher rate than the desktop customers who saw the accessories banner in May 2019.

## Exercise

Same question as before, but this time for April 2019 instead of May! Use a threshold of 0.05.